# <center>Proyecto 1</center>
## <center>Modelación y Simulación</center>
<center>Javier Anleu Alegría - 17149</center>

<center>Andrea Argüello - 17801</center>

Previous process

In [ ]:
import simpy
import math

# Variables del problema
M = 12 # num cajas
lambda_0 = 10 # clientes que llegan por hora
lambda_1 = 5 # tiempo despacho de cajero por hora
horas = 24

In [ ]:
# Atributos del proceso
env = simpy.Environment() #inicia simulacion
memoria=simpy.Container(env,capacity=100)
cpu=simpy.Resource(env,capacity=1)
waiting=simpy.Resource(env,capacity=1)

In [ ]:
factorial = lambda x: 1 if x==0 else x*(factorial(x-1))
poissonProcess = lambda k, lam, t: math.exp(-lam*t)*((lam*t)**k)/factorial(k)

# New process

## Librerías

In [1]:
import random
import numpy as np
import simpy

# Configuración de parámetros para la simulación
* `M`**:** Número de cajas en el banco
* `lambda_0`**:** Parámetro $\lambda$ para la distribución de Poisson
* `lambda_1`**:** Parámetro $\lambda_1$ para la distribución exponencial

In [2]:
M = 12
lambda_0 = 10
lambda_1 = 5
horas = 24

## Funciones para la generaión de variables aleatorias a partir de números aleatorios

### Distribución exponencial por Rejection-Sampling

**Función inversa de la distribución exponencial**

In [3]:
exponentialInv = lambda x,l: -np.log(1-x)/l

**Función para generar una variable perteneciente a la distribución exponencial**

In [4]:
def expSim():
    x = -1
    while x == -1:
        x_s = random.uniform(0,1)
        u = random.uniform(0,1)
        if u<exponentialInv(x_s,lambda_1)/(lambda_1*exponentialInv(x_s,1)):
            x = exponentialInv(x_s,lambda_1)
    return x

### Distribución de Poisson por el Método de Transformada Inversa

**Función para generar una variable perteneciente a la distribución de Poisson**

El algoritmo de la función está basado en el algoritmo para la generación de variables de Poisson por el método de transformada inversa descrito en: http://www.columbia.edu/~ks20/4404-Sigman/4404-Notes-ITM.pdf

In [5]:
def poissonSim():
    x = 0
    P = 1
    while P>=np.exp(-lambda_0):
        u = random.uniform(0,1)
        P = u*P
        if P>=np.exp(-lambda_0):
            x += 1
            P = 1
    return x

## Definición de objetos y funciones para la simulación

### Modelo de objetos y métodos

Clase para la generación de clientes. Esta generación sigue la distribución de Poisson con parámetro $\lambda$ (`lambda_0`).

In [6]:
def generate(env,number,counters):                   
    for i in range(number):
        c = customer(env,"Customer%02d"%(i,),counters)
        activate(c,c.visit(counters))
        t = 3600/number
        yield hold,self,t

In [7]:
def NoInSystem(R):                                                  
    """ Total number of customers in the resource R"""
    return (len(R.waitQ)+len(R.activeQ))                            

In [ ]:
class Customer(Process):
    """ Customer arrives, chooses the shortest queue
        is served and leaves
    """
        
    def visit(self,counters):       
        arrive = now()
        Qlength = [NoInSystem(counters[i]) for i in range(Nc)]      
        print ("%f %s: Here I am. %s"%(now(),self.name,Qlength))
        for i in range(Nc):                                         
            if Qlength[i] == 0 or Qlength[i] == min(Qlength):
                choice = i  # the chosen queue number                
                break
                
        yield request,self,counters[choice]
        wait = now()-arrive
        print ("%f %s: Waited %6.3f"%(now(),self.name,wait))
        tib = expSim()
        yield hold,self,tib
        yield release,self,counters[choice]

        print ("%f %s: Finished"%(now(),self.name))

In [8]:
def customer(env, name, counter):
    #"""Customer arrives, is served and leaves."""
    arrive = env.now
    print('%7.4f %s: Here I am' % (arrive, name))
    Qlength = [NoInSystem(counters[i]) for i in range(Nc)]      
    print ("%f %s: Here I am. %s"%(now(),name,Qlength))
    
    for i in range(M):                                         
        if Qlength[i] == 0 or Qlength[i] == min(Qlength):
            choice = i  # the chosen queue number                
            break
    
    yield Resource.request,counters[choice]
    wait = now()-arrive
    print ("%f %s: Waited %6.3f"%(now(),name,wait))
    tib = expSim()
    yield hold,env.timeout(tib)
    yield release,counters[choice]

    print ("%f %s: Finished"%(now(),name))

In [9]:
## Experiment data -------------------------
maxTime = 18000.0 # minutes
theseed = 12345                                           
                                    
## Model/Experiment ------------------------------

random.seed(theseed)
env = simpy.Environment()
kk=[simpy.Resource(env, capacity=1) for i in range(M)]   
proc = env.process(generate(env,number=poissonSim(),counters=kk))
env.run(until=maxTime)

NameError: name 'activate' is not defined